In [1]:
from torchvision import models
mobilenet = models.mobilenet_v3_small(pretrained=True)
print(mobilenet)

C:\Users\aidan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aidan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\aidan/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth


100.0%


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# download the Fashion-MNIST dataset
train_set = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_set = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

# prepare the data
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)



torch.Size([60000, 28, 28])


In [ ]:
import torch
from datetime import datetime

def training_setup(model):
    model.classifier[3] = torch.nn.Linear(1024, 10)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # record current time
    start_time = datetime.now()

    print("Parameter count:", sum(p.numel() for p in model.parameters() if p.requires_grad))

    for epoch in range(10):
        model.train()

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    end_time = datetime.now()
    elapsed = end_time - start_time
    print(f"Training completed in: {elapsed} seconds ({elapsed/10} per epoch)")

    model.eval()

    total_correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            total_correct += (outputs.argmax(dim=1) == labels).sum().item() 

    accuracy = total_correct / len(test_set)
    
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
        

In [13]:
from torchvision import models

mobilenet = models.mobilenet_v3_small(pretrained=True)

# pretrained model training
training_setup(mobilenet)


Parameter count: 1528106


KeyboardInterrupt: 